>In this area, there are sixteen memory banks; each memory bank can hold any number of blocks. The goal of the reallocation routine is to balance the blocks between the memory banks.

>The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

>The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

>For example, imagine a scenario with only four memory banks:

> - The banks start with 0, 2, 7, and 0 blocks. The third bank has the most blocks, so it is chosen for redistribution.
> - Starting with the next bank (the fourth bank) and then continuing to the first bank, the second bank, and so on, the 7 blocks are spread out over the memory banks. The fourth, first, and second banks get two blocks each, and the third bank gets one back. The final result looks like this: 2 4 1 2.
> - Next, the second bank is chosen because it contains the most blocks (four). Because there are four memory banks, each gets one block. The result is: 3 1 2 3.
> - Now, there is a tie between the first and fourth memory banks, both of which have three blocks. The first bank wins the tie, and its three blocks are distributed evenly over the other three banks, leaving it with none: 0 2 3 4.
> - The fourth bank is chosen, and its four blocks are distributed such that each of the four banks receives one: 1 3 4 1.
> - The third bank is chosen, and the same thing happens: 2 4 1 2.

> At this point, we've reached a state we've seen before: 2 4 1 2 was already seen. The infinite loop is detected after the fifth block redistribution cycle, and so the answer in this example is 5.

>Given the initial block counts in your puzzle input, *how many redistribution cycles* must be completed before a configuration is produced that has been seen before?

In [89]:
TEST_DATA_INITIAL_STATE = [0,2,7,0]
TEST_DATA_INITIAL_STATE_MAX_BLOCKS = 7
TEST_DATA_INITITAL_STATE_MAX_BLOCKS_INDEX = 2 # The third bank
TEST_DATA_SECOND_STATE = [2, 4, 1, 2]


In [106]:
def first_max(input_list):
    list_max = max(input_list)
    list_max_index = len(input_list)
    
    for index, item in enumerate(input_list):
            if item == list_max:
                if index < list_max_index:
                    list_max = item
                    list_max_index = index
                    
    return (list_max, list_max_index)


In [97]:
block_size, index = first_max(TEST_DATA_INITIAL_STATE)
print('Given the initial list the maximum block size is: {} and the index position is {}'.format(block_size, 
                                                                                                index))
print('Given the initial list the maximum block size is correct: {} and the index position is correct {}'.format(block_size == TEST_DATA_INITIAL_STATE_MAX_BLOCKS, 
                                                                                                index == TEST_DATA_INITITAL_STATE_MAX_BLOCKS_INDEX))

7 2
Given the initial list the maximum block size is: 7 and the index position is 2
Given the initial list the maximum block size is correct: True and the index position is correct True


In [98]:
import copy 

def redistribute_blocks(input_list, block_to_redistribute): 
    redistributed_list = copy.deepcopy(input_list)
    length_of_list = len(redistributed_list)
    
    index = block_to_redistribute + 1
    
    if index == length_of_list: 
        index = 0 
        
    blocks_to_redistribute = redistributed_list[index - 1]
    
    redistributed_list[index - 1] = 0 
    
    while blocks_to_redistribute > 0:
        redistributed_list[index] = redistributed_list[index] + 1
        
        index = index + 1
        if index == length_of_list: 
            index = 0
            
        blocks_to_redistribute = blocks_to_redistribute - 1
        
    return redistributed_list

In [99]:
block_size, index = last_max(TEST_DATA_INITIAL_STATE)

redistributed_list = []
redistributed_list = redistribute_blocks(TEST_DATA_INITIAL_STATE, index)

print('Given the initial list {} the next step looks like {}'.format(TEST_DATA_INITIAL_STATE,
                                                                    redistributed_list))
print('The redistributed list matches the expected next list: {}'.format(redistributed_list == TEST_DATA_SECOND_STATE))

Given the initial list [0, 2, 7, 0] the next step looks like [2, 4, 1, 2]
The redistributed list matches the expected next list: True


In [104]:
import copy 

def has_list_been_seen(list_of_lists, comparison):
    found = False
    
    for item in list_of_lists:
        if item == comparison:
            found = True 
            
    return found

def rearrange_all_blocks(input_list): 
    lists_seen = []
    list_seen = False
    steps = 0
    
    redistributed_list = copy.deepcopy(input_list)
    lists_seen.append(input_list)
    
    while not list_seen:
    
        _, index = first_max(redistributed_list)
        redistributed_list = redistribute_blocks(redistributed_list, index)

        if has_list_been_seen(lists_seen, redistributed_list):
            list_seen = True
    
        lists_seen.append(redistributed_list)
    
        steps = steps + 1
    
    return (steps, redistributed_list)

In [107]:
TEST_DATA_INITIAL_STATE = [0,2,7,0]
TEST_DATA_INITIAL_STATE_MAX_BLOCKS = 7
TEST_DATA_INITITAL_STATE_MAX_BLOCKS_INDEX = 2 # The third bank
TEST_DATA_SECOND_STATE = [2, 4, 1, 2]

steps, redistributed_list = rearrange_all_blocks(TEST_DATA_INITIAL_STATE)
print('For the input list I take {} steps to end with this list: {}'.format(steps, 
                                                                       redistributed_list))

For the input list I take 5 steps to end with this list: [2, 4, 1, 2]


In [108]:
PUZZLE_DATA = [2,8,8,5,4,2,3,1,5,5,1,2,15,13,5,14]
steps, redistributed_list = rearrange_all_blocks(PUZZLE_DATA)
print('It takesn {} steps to rearrange the puzzle data before an existing state is seen.'.format(steps))

It takesn 3156 steps to rearrange the puzzle data before an existing state is seen.


> Out of curiosity, the debugger would also like to know the size of the loop: starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

> In the example above, 2 4 1 2 is seen again after four cycles, and so the answer in that example would be 4.

> How many cycles are in the infinite loop that arises from the configuration in your puzzle input?

In [211]:
import copy 

def has_list_been_seen(list_of_lists, comparison):
    found = False
    
    for item in list_of_lists:
        if item == comparison:
            found = True 
            
    return found

def has_list_been_seen_more_than_twice(list_of_lists, comparison):
    found = False
    
    for item in list_of_lists:
        if item == comparison:
  #          print('I have seen this list')
  #          print(list_of_lists.count(item))
            if list_of_lists.count(item) >= 2:
 #               print('more than and equal to twice')
                found = True 
            
    return found

def rearrange_all_blocks_star2(input_list, lists_seen=[]): 
    list_seen = False
    steps = 0
    
    redistributed_list = copy.deepcopy(input_list)
    lists_seen.append(input_list)
    
    while not list_seen:
    
        _, index = first_max(redistributed_list)
        redistributed_list = redistribute_blocks(redistributed_list, index)

        if has_list_been_seen_more_than_twice(lists_seen, redistributed_list):
            list_seen = True
    
        lists_seen.append(redistributed_list)
    
        steps = steps + 1
    
#   print('foobar')
    return (steps, redistributed_list, lists_seen)

def find_and_count_secondary_match(input_list): 
    steps = 0 
#    print(input_list)
    steps_to_find, repeated_list, found_lists = rearrange_all_blocks(input_list)
#    print(repeated_list, steps_to_find)
    
    steps_to_second_match = 0
    second_match_found = False
    second_match_list = copy.deepcopy(repeated_list)
    
    while not second_match_found:         
        steps , second_match_list, found_lists = rearrange_all_blocks_star2(second_match_list, found_lists)
#        print(steps, 'steps to find match2')
#        print(second_match_list, 'rearrange')
        if second_match_list == repeated_list:
#            print(steps, 'hit!')
            second_match_found = True
#            if found_lists.count(second_match_list) == 2:
#        second_match_found = True
        
        found_lists.append(second_match_list)
        steps_to_second_match = steps_to_second_match + 1
        
#    print(found_lists)
    return steps
    


In [213]:
TEST_DATA_INITIAL_STATE = [0,2,7,0]
TEST_DATA_INITIAL_STATE_MAX_BLOCKS = 7
TEST_DATA_INITITAL_STATE_MAX_BLOCKS_INDEX = 2 # The third bank
TEST_DATA_SECOND_STATE = [2, 4, 1, 2]

steps = find_and_count_secondary_match(TEST_DATA_INITIAL_STATE)
print('From the first repeated list it takes {} steps to find it again'.format(steps))

From the first repeated list it takes 4 steps to find it again


In [216]:
PUZZLE_DATA = [2,8,8,5,4,2,3,1,5,5,1,2,15,13,5,14]
steps = find_and_count_secondary_match(PUZZLE_DATA)
print('It takesn {} steps to rearrange the puzzle data before an existing state is seen a second time.'.format(steps))

It takesn 1610 steps to rearrange the puzzle data before an existing state is seen a second time.
